# add water mark on all frames.

In [ ]:
import cv2
import numpy as np

original_video_path = "./OriginalVideo/videoplayback.mp4"
output_visible_video_path = "./visible_watermarked/output_visible_watermarked.mp4"
output_invisible_video_path = "./invisible_watermarked/output_invisible_watermarked.mp4"
watermark_text = "Watermark"

def add_visible_watermarks(frame, text):
    h, w, _ = frame.shape
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2
    color = (255, 255, 255)  # White color

    watermark_bg = np.zeros_like(frame)
    positions = []
    for i in range(4):
        y_pos = int(h * (0.1 + 0.2 * i))
        for j in range(3):
            x_pos = int(w * (0.1 + 0.3 * j)) 
            positions.append((x_pos, y_pos))
    for pos in positions:
        cv2.putText(watermark_bg, text, pos, font, font_scale, color, thickness, cv2.LINE_AA)

    alpha = 0.3 
    cv2.addWeighted(watermark_bg, alpha, frame, 1 - alpha, 0, frame)

    return frame

def add_invisible_watermark(frame, text):
    h, w, _ = frame.shape
    blank_image = np.zeros((h, w), dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 3
    thickness = 5
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    text_x = (w - text_size[0]) // 2
    text_y = (h + text_size[1]) // 2

    for i in range(4):
        y_offset = int(h * (0.1 + 0.2 * i))  
        for j in range(3):
            x_offset = int(w * (0.1 + 0.3 * j))
            cv2.putText(blank_image, text, (x_offset, y_offset), font, font_scale, 255, thickness, cv2.LINE_AA)

    frame[:, :, 0] = np.bitwise_and(frame[:, :, 0], 0xFE) | (blank_image >> 7)

    return frame

def process_video(video_path, visible_out, invisible_out):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    visible_writer = cv2.VideoWriter(visible_out, fourcc, fps, (width, height))
    invisible_writer = cv2.VideoWriter(invisible_out, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        visible_frame = add_visible_watermarks(frame.copy(), watermark_text)
        invisible_frame = add_invisible_watermark(frame.copy(), watermark_text)
        visible_writer.write(visible_frame)
        invisible_writer.write(invisible_frame)
    
    cap.release()
    visible_writer.release()
    invisible_writer.release()

if __name__ == "__main__":
    process_video(
        original_video_path,
        output_visible_video_path,
        output_invisible_video_path
    )

    print("Watermarking completed. Videos saved.")


Watermarking completed. Videos saved.


# add water in seclected frames with random places

In [15]:
import cv2
import numpy as np
import random

original_video_path = "./OriginalVideo/videoplayback.mp4"
output_visible_video_path = "./visible_watermarked/output_visible_watermarked.mp4"
output_invisible_video_path = "./invisible_watermarked/output_invisible_watermarked.mp4"
watermark_text = "Watermark"

def add_visible_watermarks(frame, text, opacity=0.5):
    h, w, _ = frame.shape
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2
    color = (255, 255, 255) 

    overlay = frame.copy()

    for _ in range(8):
        x_pos = random.randint(0, w - 100) 
        y_pos = random.randint(0, h - 30) 
        cv2.putText(overlay, text, (x_pos, y_pos), font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.addWeighted(overlay, opacity, frame, 1 - opacity, 0, frame)

    return frame

def add_invisible_watermark(frame, text):
    h, w, _ = frame.shape
    blank_image = np.zeros((h, w), dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 3
    thickness = 5
    for _ in range(8):
        x_pos = random.randint(0, w - 100) 
        y_pos = random.randint(0, h - 30)
        cv2.putText(blank_image, text, (x_pos, y_pos), font, font_scale, 255, thickness, cv2.LINE_AA)

    frame[:, :, 0] = np.bitwise_and(frame[:, :, 0], 0xFE) | (blank_image >> 7)

    return frame

def process_video(video_path, visible_out, invisible_out, opacity=0.5):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    visible_writer = cv2.VideoWriter(visible_out, fourcc, fps, (width, height))
    invisible_writer = cv2.VideoWriter(invisible_out, fourcc, fps, (width, height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    watermark_frames = set([0, 1, frame_count - 2, frame_count - 1])
    random_frames = random.sample(range(2, frame_count - 2), 3)
    watermark_frames.update(random_frames)

    current_frame = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if current_frame in watermark_frames:
            visible_frame = add_visible_watermarks(frame.copy(), watermark_text, opacity)
            invisible_frame = add_invisible_watermark(frame.copy(), watermark_text)
        else:
            visible_frame = frame.copy()
            invisible_frame = frame.copy()

        visible_writer.write(visible_frame)
        invisible_writer.write(invisible_frame)

        current_frame += 1

    cap.release()
    visible_writer.release()
    invisible_writer.release()

if __name__ == "__main__":
    process_video(
        original_video_path,
        output_visible_video_path,
        output_invisible_video_path,
        opacity=0.3 
    )

    print("Watermarking completed. Videos saved.")


Watermarking completed. Videos saved.


# add water in seclected frames with row wise

In [16]:
import cv2
import numpy as np
import random

original_video_path = "./OriginalVideo/videoplayback.mp4"
output_visible_video_path = "./visible_watermarked/output_visible_watermarked.mp4"
output_invisible_video_path = "./invisible_watermarked/output_invisible_watermarked.mp4"
watermark_text = "Watermark"

def add_visible_watermarks(frame, text, alpha=0.3):
    h, w, _ = frame.shape
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2
    color = (255, 255, 255) 

    watermark_bg = np.zeros_like(frame)

    positions = []
    for i in range(4):  # 4 rows
        y_pos = int(h * (0.1 + 0.2 * i)) 
        for j in range(3): 
            x_pos = int(w * (0.1 + 0.3 * j)) 
            positions.append((x_pos, y_pos))

    for pos in positions:
        cv2.putText(watermark_bg, text, pos, font, font_scale, color, thickness, cv2.LINE_AA)

    cv2.addWeighted(watermark_bg, alpha, frame, 1 - alpha, 0, frame)

    return frame

def add_invisible_watermark(frame, text):
    h, w, _ = frame.shape
    blank_image = np.zeros((h, w), dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 3
    thickness = 5

    positions = []
    for i in range(4): 
        y_pos = int(h * (0.1 + 0.2 * i)) 
        for j in range(3):  
            x_pos = int(w * (0.1 + 0.3 * j))
            positions.append((x_pos, y_pos))

    for pos in positions:
        cv2.putText(blank_image, text, pos, font, font_scale, 255, thickness, cv2.LINE_AA)

    frame[:, :, 0] = np.bitwise_and(frame[:, :, 0], 0xFE) | (blank_image >> 7)

    return frame

def process_video(video_path, visible_out, invisible_out):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    visible_writer = cv2.VideoWriter(visible_out, fourcc, fps, (width, height))
    invisible_writer = cv2.VideoWriter(invisible_out, fourcc, fps, (width, height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    watermark_frames = set([0, 1, frame_count - 2, frame_count - 1])
    random_frames = random.sample(range(2, frame_count - 2), 3)
    watermark_frames.update(random_frames)

    current_frame = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if current_frame in watermark_frames:
            visible_frame = add_visible_watermarks(frame.copy(), watermark_text)
            invisible_frame = add_invisible_watermark(frame.copy(), watermark_text)
        else:
            visible_frame = frame.copy()
            invisible_frame = frame.copy()

        visible_writer.write(visible_frame)
        invisible_writer.write(invisible_frame)

        current_frame += 1

    cap.release()
    visible_writer.release()
    invisible_writer.release()

if __name__ == "__main__":
    process_video(
        original_video_path,
        output_visible_video_path,
        output_invisible_video_path
    )

    print("Watermarking completed. Videos saved.")


Watermarking completed. Videos saved.


In [18]:
import cv2
import numpy as np
import random
import os
import glob

# Supported video extensions
video_extensions = ['.mp4', '.mov', '.mkv', '.avi', '.flv', '.webm', '.m4v', '.3gp', '.mpeg', '.ts']

input_video_folder = "./OriginalVideo/test.ts" 
output_visible_video_path = "./visible_watermarked/output_visible_watermarked.mp4"
output_invisible_video_path = "./invisible_watermarked/output_invisible_watermarked.mp4"
watermark_text = "Watermark"

def add_visible_watermarks(frame, text, alpha=0.3):
    h, w, _ = frame.shape
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2
    color = (255, 255, 255)

    watermark_bg = np.zeros_like(frame)
    positions = []
    for i in range(4):  
        y_pos = int(h * (0.1 + 0.2 * i))
        for j in range(3):  
            x_pos = int(w * (0.1 + 0.3 * j)) 
            positions.append((x_pos, y_pos))

    for pos in positions:
        cv2.putText(watermark_bg, text, pos, font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.addWeighted(watermark_bg, alpha, frame, 1 - alpha, 0, frame)

    return frame

def add_invisible_watermark(frame, text):
    h, w, _ = frame.shape
    blank_image = np.zeros((h, w), dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 3
    thickness = 5

    positions = []
    for i in range(4): 
        y_pos = int(h * (0.1 + 0.2 * i)) 
        for j in range(3):
            x_pos = int(w * (0.1 + 0.3 * j)) 
            positions.append((x_pos, y_pos))

    for pos in positions:
        cv2.putText(blank_image, text, pos, font, font_scale, 255, thickness, cv2.LINE_AA)

    frame[:, :, 0] = np.bitwise_and(frame[:, :, 0], 0xFE) | (blank_image >> 7)

    return frame

def process_video(video_path, visible_out, invisible_out):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video file: {video_path}")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    visible_writer = cv2.VideoWriter(visible_out, fourcc, fps, (width, height))
    invisible_writer = cv2.VideoWriter(invisible_out, fourcc, fps, (width, height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    watermark_frames = set([0, 1, frame_count - 2, frame_count - 1])
    if frame_count > 4:
        random_frames = random.sample(range(2, frame_count - 2), min(3, frame_count - 4))
        watermark_frames.update(random_frames)

    current_frame = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if current_frame in watermark_frames:
            visible_frame = add_visible_watermarks(frame.copy(), watermark_text)
            invisible_frame = add_invisible_watermark(frame.copy(), watermark_text)
        else:
            visible_frame = frame.copy()
            invisible_frame = frame.copy()

        visible_writer.write(visible_frame)
        invisible_writer.write(invisible_frame)

        current_frame += 1

    cap.release()
    visible_writer.release()
    invisible_writer.release()

def find_video_file(path, extensions):
    if os.path.isfile(path): 
        return path
    elif os.path.isdir(path): 
        video_files = []
        for ext in extensions:
            video_files.extend(glob.glob(os.path.join(path, f"*{ext}")))

        if not video_files:
            raise FileNotFoundError(f"No valid video files found in {path} with extensions: {extensions}")

        return video_files[0]
    else:
        raise FileNotFoundError(f"The path {path} is neither a file nor a directory.")

if __name__ == "__main__":
    try:
        video_file = find_video_file(input_video_folder, video_extensions)

        process_video(
            video_file,
            output_visible_video_path,
            output_invisible_video_path
        )

        print(f"Watermarking completed. Videos saved.\nProcessed video: {video_file}")
    except Exception as e:
        print(f"Error: {e}")


Watermarking completed. Videos saved.
Processed video: ./OriginalVideo/test.ts


In [ ]:
import cv2
import numpy as np
import random
import os
import glob
from scipy.fft import dct, idct

video_extensions = ['.mp4', '.mov', '.mkv', '.avi', '.flv', '.webm', '.m4v', '.3gp', '.mpeg', '.ts']

input_video_folder = "./OriginalVideo/test.ts"
output_visible_video_path = "./visible_watermarked/output_visible_watermarked.mp4"
output_invisible_video_path = "./invisible_watermarked/output_invisible_watermarked.mp4"
watermark_text = "Watermark"
watermark_strength = 2 

def embed_watermark_dct(block, watermark_matrix, strength=1):
    dct_block = dct(dct(block.astype(float), axis=0, norm='ortho'), axis=1, norm='ortho')

    h, w = watermark_matrix.shape
    dct_block[0:h, 0:w] = dct_block[0:h, 0:w] + strength * watermark_matrix

    watermarked_block = idct(idct(dct_block, axis=0, norm='ortho'), axis=1, norm='ortho').real
    return watermarked_block.astype(np.uint8)
def create_watermark_matrix(text, block_size):

    h, w = block_size
    blank_image = np.zeros((h, w), dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    thickness = 1
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]

    text_x = (w - text_size[0]) // 2
    text_y = (h + text_size[1]) // 2

    cv2.putText(blank_image, text, (text_x,text_y), font, font_scale, 255, thickness, cv2.LINE_AA)
    
    return blank_image

def add_visible_watermarks(frame, text, alpha=0.3):
    h, w, _ = frame.shape
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2
    color = (255, 255, 255)

    watermark_bg = np.zeros_like(frame)
    positions = []
    for i in range(4):  
        y_pos = int(h * (0.1 + 0.2 * i))
        for j in range(3):  
            x_pos = int(w * (0.1 + 0.3 * j)) 
            positions.append((x_pos, y_pos))

    for pos in positions:
        cv2.putText(watermark_bg, text, pos, font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.addWeighted(watermark_bg, alpha, frame, 1 - alpha, 0, frame)

    return frame

def add_invisible_watermark(frame, text, strength=1):
    h, w, _ = frame.shape
    block_size = (16, 16) 

    watermark_matrix = create_watermark_matrix(text, block_size)

    for y in range(0, h, block_size[0]):
        for x in range(0, w, block_size[1]):
            block = frame[y:y + block_size[0], x:x + block_size[1], 0]
            
            if block.shape == block_size :
                frame[y:y + block_size[0], x:x + block_size[1], 0] = embed_watermark_dct(block, watermark_matrix, strength)

    return frame
    

def process_video(video_path, visible_out, invisible_out):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video file: {video_path}")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    visible_writer = cv2.VideoWriter(visible_out, fourcc, fps, (width, height))
    invisible_writer = cv2.VideoWriter(invisible_out, fourcc, fps, (width, height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    watermark_frames = set([0, 1, frame_count - 2, frame_count - 1])
    if frame_count > 4:
        random_frames = random.sample(range(2, frame_count - 2), min(3, frame_count - 4))
        watermark_frames.update(random_frames)

    current_frame = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if current_frame in watermark_frames:
            visible_frame = add_visible_watermarks(frame.copy(), watermark_text)
            invisible_frame = add_invisible_watermark(frame.copy(), watermark_text, watermark_strength)
        else:
            visible_frame = frame.copy()
            invisible_frame = frame.copy()

        visible_writer.write(visible_frame)
        invisible_writer.write(invisible_frame)

        current_frame += 1

    cap.release()
    visible_writer.release()
    invisible_writer.release()


def find_video_file(path, extensions):
    if os.path.isfile(path):
        return path
    elif os.path.isdir(path):
        video_files = []
        for ext in extensions:
            video_files.extend(glob.glob(os.path.join(path, f"*{ext}")))

        if not video_files:
            raise FileNotFoundError(f"No valid video files found in {path} with extensions: {extensions}")
        return video_files[0]
    else:
        raise FileNotFoundError(f"The path {path} is neither a file nor a directory.")

if __name__ == "__main__":
    try:
        video_file = find_video_file(input_video_folder, video_extensions)

        process_video(
            video_file,
            output_visible_video_path,
            output_invisible_video_path
        )

        print(f"Watermarking completed. Videos saved.\nProcessed video: {video_file}")
    except Exception as e:
        print(f"Error: {e}")

Watermarking completed. Videos saved.
Processed video: ./OriginalVideo/test.ts
